# Adapters in Practice — from Decision to Deployment

This notebook follows **Lesson 5** in order: decision ladder → data schema → baseline prompt → LoRA adapters → QLoRA → validation → packaging/rollback → run cards. It is designed to run on a single GPU (12–16 GB VRAM).

In [1]:
# 0) Preamble & installs (Colab/clean env)
!pip -q install -U transformers peft datasets accelerate bitsandbytes evaluate rank_bm25

import os, re, json, random, math, textwrap
import torch
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer, pipeline)
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from evaluate import load as load_metric

print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CPU


## 1) Task & data: one schema + fixed dev set
We'll create a small synthetic dataset with the schema:
**Instruction / Input / Expected Response**. Keep a **fixed dev set** for validation only.

In [2]:
# Build a tiny synthetic dataset in the class schema
raw = [
  {"input": 'vpn down hr floor—users can’t log in', "label": "Network", "title_clean": "HR floor VPN outage—login failures"},
  {"input": 'reset password portal loop', "label": "Account", "title_clean": "Password reset portal loop"},
  {"input": 'email bounce for vendors', "label": "Email", "title_clean": "Vendor email bounces"},
  {"input": 'wifi flaky conference room b', "label": "Network", "title_clean": "Conference Room B Wi-Fi unstable"},
  {"input": 'new hire cannot access jira', "label": "Access", "title_clean": "New hire cannot access Jira"},
]

def jitter(s):
    return s.replace("—"," - ")

data = []
for r in raw:
    for _ in range(40):  # ~200 examples total
        data.append({
            "instruction": "Convert this note to a clean ticket title and label.",
            "input": jitter(r["input"]),
            "expected": f'{r["title_clean"]} || Label: {r["label"]}'
        })
random.shuffle(data)
ds = Dataset.from_list(data).train_test_split(test_size=0.15, seed=7)
dev = ds["test"]   # fixed dev set
train = ds["train"]
len(train), len(dev)

(170, 30)

### Simple validation helpers
**Format-pass** (does the output follow `<title> || Label: <LABEL>`?) and a rough **label-accuracy proxy**.

In [3]:
def parse_pred(pred:str):
    m = re.match(r'(.+?)\s*\|\|\s*Label:\s*([A-Za-z]+)', pred.strip())
    return {"format_ok": bool(m), "label": (m.group(2) if m else None)}

def evaluate_preds(preds, refs):
    ok = 0; label_ok = 0
    for p, r in zip(preds, refs):
        pp = parse_pred(p)
        ok += pp["format_ok"]
        label_ok += int(pp["label"] in r["expected"])  # proxy: label string present
    n = len(preds)
    return {"format_pass": ok/n, "label_acc_proxy": label_ok/n}

## 2) Baseline: Prompt-only (optionally with tiny retrieval)
Start with the lightest approach and measure.

In [4]:
gen = pipeline("text-generation", model="sshleifer/tiny-gpt2", tokenizer="sshleifer/tiny-gpt2", device_map="auto")

prompt_tmpl = (
    "You are a helpful assistant.\n"
    "Instruction: Convert this note to a clean ticket title and label.\n"
    "Input: {inp}\n"
    "Respond EXACTLY like: <title> || Label: <OneWordLabel>\n"
)

def gen_baseline(batch):
    outs = []
    for x in batch["input"]:
        text = gen(prompt_tmpl.format(inp=x), max_new_tokens=32, do_sample=False)[0]["generated_text"]
        outs.append(text.splitlines()[-1])  # take last line
    return outs

baseline_preds = gen_baseline(dev)
evaluate_preds(baseline_preds, dev)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/452k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

ValueError: Could not load model sshleifer/tiny-gpt2 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>, <class 'transformers.models.gpt2.modeling_tf_gpt2.TFGPT2LMHeadModel'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/Users/jerhenry/Documents/Workdoc/f-Virtual_Machines/Environments/Pytorch_nov2024/lib/python3.11/site-packages/transformers/pipelines/base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jerhenry/Documents/Workdoc/f-Virtual_Machines/Environments/Pytorch_nov2024/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    ret

## 3) LoRA adapters: Q & V on a few layers (beginner defaults)
Train a **tiny** LoRA head to stabilize format and labels. **LR = learning rate** here; **rank (r)** is the LoRA size.

In [ ]:
BASE = "sshleifer/tiny-gpt2"  # tiny for demo; swap for a small chat model on better hardware
tok = AutoTokenizer.from_pretrained(BASE)
model = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto")

def to_text(example):
    return f"Instruction: {example['instruction']}\nInput: {example['input']}\nExpected Response:"

train_texts = [to_text(x) for x in train]
dev_texts = [to_text(x) for x in dev]
labels = [x["expected"] for x in train]
labels_dev = [x["expected"] for x in dev]

def format_for_causal_lm(prompts, targets, tok, max_len=256):
    enc = tok(prompts, padding=True, truncation=True, return_tensors="pt", max_length=max_len)
    with tok.as_target_tokenizer():
        lab = tok(targets, padding=True, truncation=True, return_tensors="pt", max_length=max_len)
    X, Y = [], []
    for p, t in zip(enc["input_ids"], lab["input_ids"]):
        x = torch.cat([p, t], dim=0)[:max_len]
        y = torch.cat([torch.full_like(p, -100), t], dim=0)[:max_len]
        X.append(x); Y.append(y)
    return Dataset.from_dict({"input_ids": X, "labels": Y})

train_ds = format_for_causal_lm(train_texts, labels, tok)
dev_ds   = format_for_causal_lm(dev_texts, labels_dev, tok)

# LoRA config — tiny-gpt2 uses c_attn (qkv together). For LLaMA-like models use ["q_proj","v_proj"].
lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["c_attn"],  # Q/V (and K) projection in this tiny model
    bias="none", task_type="CAUSAL_LM"
)
lora_model = get_peft_model(model, lora_cfg)
lora_model.print_trainable_parameters()

In [ ]:
# Training with gradient clipping and simple evaluation schedule
args = TrainingArguments(
    output_dir="out_lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-4,   # LR = learning rate
    num_train_epochs=2,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    max_grad_norm=1.0,
    fp16=True if torch.cuda.is_available() else False,
)

def data_collator(features):
    import torch
    return {k: torch.nn.utils.rnn.pad_sequence([f[k] for f in features], batch_first=True, padding_value=tok.pad_token_id)
            for k in ["input_ids","labels"]}

trainer = Trainer(model=lora_model, args=args, train_dataset=train_ds, eval_dataset=dev_ds, data_collator=data_collator)
trainer.train()

### Quick validation (format-pass & label proxy)

In [ ]:
def gen_with_adapter(model, prompts):
    outs = []
    for p in prompts:
        ids = tok(p, return_tensors="pt").to(model.device)
        g = model.generate(**ids, max_new_tokens=32, do_sample=False)
        text = tok.decode(g[0], skip_special_tokens=True)
        outs.append(text.split("Expected Response:")[-1].strip().splitlines()[0])
    return outs

preds = gen_with_adapter(trainer.model, dev_texts)
evaluate_preds(preds, dev)

## 4) QLoRA: fit in smaller VRAM (4-bit base + FP16/BF16 adapters)

In [ ]:
from transformers import BitsAndBytesConfig

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else None,
)

q_model = AutoModelForCausalLM.from_pretrained(BASE, quantization_config=bnb, device_map="auto")
q_model = prepare_model_for_kbit_training(q_model)

q_lora_cfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, target_modules=["c_attn"], bias="none", task_type="CAUSAL_LM")
q_lora = get_peft_model(q_model, q_lora_cfg)
# (Optional) Repeat a very small training loop like above if time permits

## 5) Knob ablation: rank (r)=8 vs 16 — change one thing at a time

In [ ]:
def run_one(r):
    model = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto")
    cfg = LoraConfig(r=r, lora_alpha=16, lora_dropout=0.05, target_modules=["c_attn"], bias="none", task_type="CAUSAL_LM")
    m = get_peft_model(model, cfg)
    tr = Trainer(model=m, args=args, train_dataset=train_ds, eval_dataset=dev_ds, data_collator=data_collator)
    tr.train()
    preds = gen_with_adapter(tr.model, dev_texts)
    return r, evaluate_preds(preds, dev)

for r in [8, 16]:
    print(run_one(r))

## 6) Packaging & rollback: save only the adapter

In [ ]:
# Save the adapter and tokenizer
trainer.model.save_pretrained("adapter_v1")
tok.save_pretrained("adapter_v1")

# Load in a fresh session and apply the adapter
base = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto")
adapted = PeftModel.from_pretrained(base, "adapter_v1")
# Rollback = just use `base` without loading the adapter

## 7) Run Card (log your experiment)

In [ ]:
run_card = {
  "base": BASE,
  "adapter": {"r": 8, "alpha": 16, "targets": ["c_attn (Q/V)" ]},
  "learning_rate": 1e-4,  # LR = learning rate
  "dev_metrics": evaluate_preds(preds, dev),
  "decision": "keep / try higher rank",
  "notes": "schema clean, masks ok"
}
with open("run_card.json","w") as f:
    json.dump(run_card, f, indent=2)
run_card

## 8) Optional: Pick‑a‑Path presets (for classroom toggles)

In [ ]:
paths = {
 "ticket_tidy": {"strategy":"adapters", "r":8, "targets":["qv"], "metric":"format_pass"},
 "policy_qa":   {"strategy":"prompt_rag", "few_shot":3, "citation":True},
 "brand_style": {"strategy":"adapters", "r":16, "targets":["qv","o"], "metric":"style_match"}
}
print(json.dumps(paths, indent=2))